<h1, align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;数据科学引论 - Python之道 </h1>

<h1, align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;第9课 机器学习</h1>
# <center>分类与聚类</center>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2>目录</h2>  
<font size = 3>
1. <a href="#ref0">将菜谱数据导入Python</a>  
2. <a href="#ref1">了解数据 & 数据准备</a>  
3. <a href="#ref2">哪几种烹饪方法是最相似的? (k-means聚类)</a>  
4. <a href="#ref3">我们能否根据食材来预测烹饪? (决策树)</a>  
5. <a href="#ref4">模型评估</a>  
 
</font>
<br>
<p></p>
</div>

# <center> 1. 将菜谱数据导入Python</center>

#### 轮到你了:

将数据导入python:

> `recipes = pd.read_csv('recipes.csv')`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import io
from scipy import misc
from pylab import rcParams
%matplotlib inline
plt.style.use('ggplot')
#linux !wget  -O recipes.csv https://ibm.box.com/shared/static/5wah9atr5o1akuuavl2z9tkjzdinr1lv.csv 
recipes = pd.read_csv('resources/data/recipes.csv')
print ("Done")

<hr><a id="ref1"></a>

<h1 align = center> 2. 了解数据 & 数据准备 </h1>
<br>
<br>
<img src = https://ibm.box.com/shared/static/ctv4qau0q7ny0af4jp8mwi50l8fehmsz. width = 600>

### 显示前几行

In [ ]:
recipes.head()

In [ ]:
recipes[recipes['jasmine_tea']=='Yes']

#### 总共有多少列，多少行?
(rows, columns)

In [ ]:
recipes.shape

In [ ]:
recipes.columns

<br>
#### 哪个国家菜谱数量最多?
首先我们还是先要看下数据是否需要清洗。

In [ ]:
#frequency table
recipes['country'].value_counts()

### 问题: 国家名不一致 (大小写与别名)

**目标:**
- 把所有名字变成小写
- 使国家名一致

把所有名字变成小写:

In [ ]:
recipes['country'] = recipes['country'].str.lower()

使国家名一致:

In [ ]:
recipes.replace(to_replace = "china", value = "chinese", inplace=True)
recipes.replace(to_replace = "france", value = "french", inplace=True)
recipes.replace(to_replace = "germany", value = "german", inplace=True)
recipes.replace(to_replace = "india", value = "indian", inplace=True)
recipes.replace(to_replace = "israel", value = "jewish", inplace=True)
recipes.replace(to_replace = "italy", value = "italian", inplace=True)
recipes.replace(to_replace = "japan", value = "japanese", inplace=True)
recipes.replace(to_replace = "korea", value = "korean", inplace=True)
recipes.replace(to_replace = "mexico", value = "mexican", inplace=True)
recipes.replace(to_replace = "scandinavia", value = "scandinavian", inplace=True)
recipes.replace(to_replace = "thailand", value = "thai", inplace=True)
recipes.replace(to_replace = "vietnam", value = "vietnamese", inplace=True)

重新观察数据:

In [ ]:
#frequency table
recipes['country'].value_counts()

### 问题: 某些国家的菜谱极少
删除这些菜谱数量少于50的国家。

In [ ]:
before =recipes.shape[0] #number of rows of original df
HighReceipesCountries=recipes['country'].value_counts()>50
recipes = (recipes[
        recipes['country'].isin(
            HighReceipesCountries[HighReceipesCountries].index.tolist()
        )
    ])
after=recipes.shape[0];
#python2 print before - after, "rows removed."

print (before - after, "rows removed.")
recipes['country'].value_counts()

In [ ]:
HighReceipesCountries.values

### 问题: 如果有一种食物含有大米（rice）、酱油（soy_sauce）、芥末（wasabi）和海藻（seaweed），我们能不能说它就是日本的食物？ 

In [ ]:
recipes[(recipes['rice' ]== "Yes" ) &
   (recipes['soy_sauce']=="Yes") & 
   (recipes['wasabi']== "Yes") &
   (recipes['seaweed']=="Yes")]

 <br> 
### 问题: 哪些食材是最常见的?哪些最不常见?

**目的:**
- 统计所有菜谱的食材

In [ ]:
CountIngredients=(recipes=='Yes').sum()
CountIngredients.sort_values(ascending=False)

In [ ]:
(recipes=='Yes').head()

In [ ]:
np.array([True, True, True]).sum()

## 上述问题都是针对所有食谱的，那么每个国家使用食材的情况如何呢？

<hr>

### 问题: 不同国家的食材分布情况有何不同?

每个国家的“概况”是什么??  
- 中国人通常用什么食材?  
- 到底什么是“加拿大食物”?  

**目标:**
- 统计每个国家的每种食材数量，然后根据该国家的菜谱数进行归一化
    - 在结果数据集中，每个国家占一行，每种食材占一列
    - 对于每个国家每种食材，展示含有某种食材的菜谱的占比
    - 把结果导出到一个新的数据集
    - 找到每个国家最喜欢使用的食材

In [ ]:
(recipes=='Yes').head()

In [ ]:
by_country_aggr=(recipes=='Yes').groupby(recipes['country']).sum()
by_country_normalized = by_country_aggr.div(by_country_aggr.sum(axis=1), axis=0)
by_country_normalized.head()

### 什么是中国菜?

In [ ]:
by_country_normalized.loc['chinese'].sort_values( ascending=False)
#by_country_norm.ix['canada',:].sort_values( ascending=False)


In [ ]:
#Try for different  regions 
NameofRegion=by_country_normalized.index.get_values()
NameofRegion

### 问题: 在每个国家，最受欢迎的五种食材是什么呢?

**目标:**
- 对每个国家，获取最常见的5种食材

<br>

In [ ]:
for Region in NameofRegion:
#python2     print Region
#python2     print by_country_normalized.loc[Region].sort_values( ascending=False).ix[0:5] 
#python2     print "\n"
    print (Region)
    print (by_country_normalized.loc[Region].sort_values( ascending=False).ix[0:5]) 
    print ("\n")

<hr><a id="ref2"></a>

# <center>3. 哪些烹饪方法是最相似的? (K-means聚类)</center>

从sklearn加载k-means聚类库

In [ ]:
from sklearn.cluster import KMeans

设置簇的个数:

In [ ]:
n_cluster = 5

运行聚类:

In [ ]:
y_pred = KMeans(n_clusters = n_cluster, random_state=0).fit_predict(by_country_normalized)

## 展示聚类结果 

In [ ]:
for i in range(0,n_cluster):
    print ("Cluster number "+str(i) +"----------------------------------------------------------------------------")
    print (list(by_country_normalized[y_pred==i].index))
    print ("")

<hr>

<img src = https://ibm.box.com/shared/static/jrv1mg8v3e7gm58go1ccn4u78pxhmb1l.png width = 600>

<hr><a id="ref3"></a>

# <center> 4. 我们能否根据食材来预测烹饪类型? (决策树)</center>

**总体目标:**
- 创建一个决策树模型基于食材预测烹饪类型（国家）

<br>
**目标:**
- 加载与决策树相关的库 
- 训练所有食谱得到一棵决策树


In [ ]:
#import sklearn.cross_validation as cv
from sklearn import tree
from sklearn.metrics import accuracy_score,confusion_matrix

###  将所有列转换为数字 ，将No转换为0，Yes转换为1

In [ ]:
recipes.replace(to_replace='No', value=0, inplace=True)
recipes.replace(to_replace='Yes', value=1, inplace=True)

In [ ]:
recipes.head()

查看前五行数据中rice列的数据

In [ ]:
recipes.loc[0:5]["rice"]

### 获取每个样本的分类

In [ ]:
y = recipes['country']
y[0:10]

### 获取每个样本的特征

In [ ]:
X = recipes.ix[:,1:]
X[0:10]

<br> 
**问题:**
- 决策树不够具体 

**目标:**
- 单独为东亚国家（韩国、日本、中国、泰国）和印度训练决策树 

#### 使用指定国家的数据来生成决策树模型，并且将结果赋值给变量bamboo_tree

选择东亚国家+印度

In [ ]:
Newdata = recipes[recipes.country.isin(["korean","japanese","chinese","thai","indian"])]

选择每个样本的特征和分类

In [ ]:
y=Newdata['country']
X=Newdata.ix[:,1:]

创建并且训练分类树

In [ ]:
bamboo_tree = tree.DecisionTreeClassifier(max_depth=5,min_samples_split=100,min_samples_leaf=100)
bamboo_tree.fit(X, y)

### 树的可视化

In [ ]:
#! 管理员命令行 conda install pydotplus
# 同时在外部需要下载http://www.graphviz.org/pub/graphviz/stable/windows/graphviz-2.38.msi然后双击安装，这是一个画流程图的工具


from sklearn.externals.six import StringIO
import pydotplus
import matplotlib.image as mpimg

# 然后要在环境变量里添加这个工具的可执行目录，否则之后的代码会出现InvocationException: GraphViz's executables not found 错误
import os     
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [ ]:
dot_data = StringIO()
filename = "bombootree.png"
Features = list(recipes.ix[:,1:].columns.values)
out=tree.export_graphviz(bamboo_tree,feature_names=Features, out_file=dot_data,class_names=np.unique(y), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

执行一次预测并且得到决策树的准确率

In [ ]:
y_pred=bamboo_tree.predict(X)
accuracy_score(y, y_pred)

In [ ]:
y_pred

In [ ]:
? accuracy_score

# <center> 5. 模型评估</center>

**目标:**
- 将数据分成训练数据和测试数据两部分
- 在训练数据上训练模型
- 使用不含国家标签的测试数据去检测模型的预测结果

**将数据集限制在 东亚国家和印度**
把新的数据集赋值给`bamboo`:

In [ ]:
bamboo=recipes[recipes.country.isin(["korean","japanese","chinese","thai","indian"])]

In [ ]:
bamboo.head()

#### 每一个烹饪法（国家）都有多少种食谱?

In [ ]:
bamboo['country'].value_counts()

#### 接下来，让我们从原始数据集(`bamboo`)中创建训练集和测试集

In [ ]:
import random
random.seed(99)

testdf = pd.DataFrame()
traindf = pd.DataFrame()

msk = np.random.rand(len(bamboo)) < 0.7
Traindf = bamboo[msk]
Testdf = bamboo[~msk]

#### 确认每个烹饪法的食谱数量:

In [ ]:
#Original dataset
bamboo['country'].value_counts()

In [ ]:
#Training dataset 
Traindf['country'].value_counts()

In [ ]:
#Testing dataset 
Testdf['country'].value_counts()

#### 从训练和测试数据中选择特征和分类

In [ ]:
y_train=Traindf['country']
X_train=Traindf.ix[:,1:]

y_test=Testdf['country']
X_test=Testdf.ix[:,1:]

y_train

#### 创建分类树并对分类树进行训练

In [ ]:
bamboo_tree = tree.DecisionTreeClassifier(max_depth=5,min_samples_split=20,min_samples_leaf=100,)
bamboo_tree.fit(X_train, y_train)

### 用变量bamboo_tree在数据集bamboo_test上预测烹饪方法:

In [ ]:
y_pred=bamboo_tree.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

### 创建一个混淆矩阵
- 将模型预测的结果和实际的烹饪标签进行比较

In [ ]:
import matplotlib.ticker as ticker

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=None):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    # I also added cmap=cmap here, to make use of the 
    # colormap you specify in the function call
    cax = ax.matshow(cm,cmap=cmap)
    plt.title(title)
    fig.colorbar(cax)
    if labels:
        ax.set_xticklabels([''] + labels)
        ax.set_yticklabels([''] + labels)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    plt.xlabel('Predicted')
    plt.ylabel('True')

In [ ]:
labels = ['chinese','indian','japanese','korean','thai']
cm = confusion_matrix( y_test,y_pred)
plot_confusion_matrix(cm, labels=labels)


In [ ]:
cm

#### 表格解释:
- 每一行是实际菜谱的烹饪分类
- 从左上角到右下角的对角线显示了正确的预测
- 每个值都代表了国家X的食谱被正确地标记为国家X
- 例:148(60+3+65+11+9)个中国菜谱中有60个预测是正确的，预测结果为“中国”，另外30%的错误预测成了韩国，6.75预测成了日本。
#### 讨论:
- 哪些烹饪可能是错误分类的，哪些是正确分类的?
- 哪些烹饪方式容易被误认成其他烹饪方式?

<br>
<hr><a id="ref5"></a>

### 食谱之外

#### 现实生活中应用决策树的例子
- 客户流失
- 信用卡诈骗
- 癌症预测

#### 现实生活中运用聚类的例子
- 顾客区分
- 推荐引擎

<hr>

<hr>
**引用**  
食谱数据集来自: 
- [Ahn, Yong-Yeol, et al. "Flavor network and the principles of food pairing." Scientific reports 1 (2011).](http://yongyeol.com/papers/ahn-flavornet-2011.pdf)
- 关于食材的数据集也可以在这里找到:
  - **`recipes.csv`** (64.2 MB):  http://bit.ly/recipesdata